# **Data Load**

# 1. 라이브러리 및 데이터 불러오기

In [ ]:
!pip install langchain langchain_community chromadb openai langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2


In [ ]:
import pandas as pd
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import FewShotChatMessagePromptTemplate

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/CARD-LANGCHAIN/data/unique_clusters_df.csv')

In [ ]:
data.head()

,성별,연령,Cluster,LifeStage,Desribe
0,여성,40대,2,기혼,납부
1,남성,30대,0,기혼,"쇼핑, 교육, 의료, 여유생활, 일상생활, 요식"
2,여성,40대,0,기혼,"쇼핑, 교육, 의료, 여유생활, 일상생활, 요식"
3,남성,40대,1,기혼,교통
4,여성,50대,2,기혼,납부


In [ ]:
data['Desribe'].unique()

array(['납부', '쇼핑, 교육, 의료, 여유생활, 일상생활, 요식', '교통', '사교활동', '해외'],
      dtype=object)

In [ ]:
json_data = data.to_json(orient='records', force_ascii=False)

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

In [ ]:
# 고객 데이터 로드
loader = CSVLoader(file_path = '/content/drive/MyDrive/CARD-LANGCHAIN/data/unique_clusters_df.csv')
customer_data = loader.load()

In [ ]:
customer_data[0]

Document(metadata={'source': '/content/drive/MyDrive/CARD-LANGCHAIN/data/unique_clusters_df.csv', 'row': 0}, page_content='성별: 여성\n연령: 40대\nCluster: 2\nLifeStage: 기혼\nDesribe: 납부')

In [ ]:
OPENAI_API_KEY = ''
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [ ]:


# 고객 벡터 스토어 정의
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(
    documents = customer_data,
    embedding=embedding
)

# Retriever 생성
retriever = vectordb.as_retriever(search_kwargs={'k':5})

template = """
당신은 고객 선정 담당자입니다.
주어진 카드 혜택에 맞게 광고를 진행할 고객 선정을 하세요.
여러 고객군을 선정해도 됩니다.
고객군의 연령대는 겹치지 않게 선정하세요.
선정한 고객군을 알려주세요.
{context}
Question : {question}
Answer:
"""

prompt = PromptTemplate.from_template(template)

llm = ChatOpenAI(model_name='gpt-4o-mini',
                 streaming=True,
                 callbacks=[StreamingStdOutCallbackHandler()]
                 )

# 프롬프트 덮어쓰기
# 기존의 chatgpt의 프롬프트를 사용하는 것이 아닌, 우리가 준비한 프롬프트로 덮어쓰기
# {context}에 해당하는 조회 결과가 답변할 때 들어갈 수 있게 세팅

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type_kwargs = {'prompt':prompt},
    retriever=retriever, # 검색기의 조회 결과를 {context}에 넣어준다.
    return_source_documents=True
)
# 할인, 공과금 납부 건당 300원 할인
query = "주어진 카드 혜택 : 백화점 쇼핑 10% 할인"
chatbot_response = qa_chain(query)

<ipython-input-53-482b9d7e708a>:25: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name='gpt-4o-mini',


주어진 카드 혜택인 "백화점 쇼핑 10% 할인"에 맞춰 고객군을 선정하겠습니다. 50대 남성 고객군은 중복되므로, 각기 다른 연령대의 남성을 대상으로 선정하겠습니다.

1. **고객군 1**: 
   - 성별: 남성
   - 연령: 40대
   - Cluster: 0
   - LifeStage: 중년
   - Describe: 쇼핑, 여유생활, 일상생활, 요식

2. **고객군 2**: 
   - 성별: 남성
   - 연령: 60대
   - Cluster: 0
   - LifeStage: 노령
   - Describe: 쇼핑, 의료, 여유생활, 일상생활

3. **고객군 3**: 
   - 성별: 남성
   - 연령: 30대
   - Cluster: 0
   - LifeStage: 청년
   - Describe: 쇼핑, 여유생활, 일상생활, 요식

이렇게 총 3개의 고객군을 선정하였습니다. 각 고객군은 연령대가 겹치지 않으며, 카드 혜택을 활용할 수 있는 잠재성이 있습니다.

## Few shot prompt 추가하기

## llm chain 형성  
### 고객선정 llm chain &  광고문구 chain   
### **1. 고객선정 llm**